In [2]:
# pip install onnxruntime 
# pip install --user tf2onnx
import tensorflow as tf

from keras.losses import mse, cosine_similarity
def make_mse_cosine_loss(alpha):
    def mse_cosine_loss(y_true, y_pred):
        # y_pred = tf.clip_by_value(y_pred, clip_value_min=0, clip_value_max=127)
        return alpha * (1 * cosine_similarity(y_true, y_pred)) + (1 - alpha) * mse(y_true, y_pred)
    return mse_cosine_loss

ALPHA = 0.15
mse_cosine_loss = make_mse_cosine_loss(ALPHA)

MODEL_H5_PATH = 'saved_models/mvi-v2-2023-07-20_13-00_56-h4-e5-mse_cosine_loss-alpha0.15-m0.60-LSTM-luong_attention-MAESTRO.h5'
model = tf.keras.models.load_model(MODEL_H5_PATH, custom_objects={'mse_cosine_loss': mse_cosine_loss})

In [4]:
import tf2onnx
from pathlib import Path

onnx_model, _ = tf2onnx.convert.from_keras(model, input_signature=None)

MODEL_ONNX_PATH = f'{Path(MODEL_H5_PATH).stem}.onnx'
output_path = f'saved_models/{MODEL_ONNX_PATH}'
with open(output_path, 'wb') as f:
    f.write(onnx_model.SerializeToString())